# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 8:52AM,250782,19,TR-RN-11152022,"GCH Granules USA, Inc.",Released
1,Nov 15 2022 8:42AM,250777,12,HH-38127,Hush Hush,Released
2,Nov 15 2022 8:42AM,250777,12,HH-38128,Hush Hush,Released
3,Nov 15 2022 8:42AM,250777,12,HH-38129,Hush Hush,Released
4,Nov 15 2022 8:42AM,250777,12,HH-38130,Hush Hush,Released
5,Nov 15 2022 8:42AM,250777,12,HH-38131,Hush Hush,Released
6,Nov 15 2022 8:42AM,250777,12,HH-38132,Hush Hush,Released
7,Nov 15 2022 8:42AM,250777,12,HH-38133,Hush Hush,Released
8,Nov 15 2022 8:42AM,250777,12,HH-38135,Hush Hush,Released
9,Nov 15 2022 8:42AM,250777,12,HH-38136,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,250774,Released,1
30,250775,Released,2
31,250776,Released,62
32,250777,Released,17
33,250782,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250774,NaN,NaN,1.0
250775,NaN,NaN,2.0
250776,NaN,NaN,62.0
250777,NaN,NaN,17.0
250782,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250668,27.0,0.0,10.0
250669,0.0,0.0,4.0
250670,0.0,0.0,48.0
250674,0.0,1.0,0.0
250682,0.0,0.0,64.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250668,27,0,10
250669,0,0,4
250670,0,0,48
250674,0,1,0
250682,0,0,64


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250668,27,0,10
1,250669,0,0,4
2,250670,0,0,48
3,250674,0,1,0
4,250682,0,0,64


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250668,27,,10
1,250669,,,4
2,250670,,,48
3,250674,,1,
4,250682,,,64


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 8:52AM,250782,19,"GCH Granules USA, Inc."
1,Nov 15 2022 8:42AM,250777,12,Hush Hush
18,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation
80,Nov 15 2022 8:22AM,250775,10,ISDIN Corporation
82,Nov 15 2022 8:19AM,250774,10,ISDIN Corporation
83,Nov 15 2022 8:18AM,250773,10,ISDIN Corporation
86,Nov 15 2022 7:42AM,250772,19,"AdvaGen Pharma, Ltd"
90,Nov 15 2022 7:37AM,250771,20,"Exact-Rx, Inc."
99,Nov 15 2022 4:39AM,250769,10,ISDIN Corporation
114,Nov 14 2022 6:13PM,250768,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 8:52AM,250782,19,"GCH Granules USA, Inc.",,,1
1,Nov 15 2022 8:42AM,250777,12,Hush Hush,,,17
2,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,,,62
3,Nov 15 2022 8:22AM,250775,10,ISDIN Corporation,,,2
4,Nov 15 2022 8:19AM,250774,10,ISDIN Corporation,,,1
5,Nov 15 2022 8:18AM,250773,10,ISDIN Corporation,,,3
6,Nov 15 2022 7:42AM,250772,19,"AdvaGen Pharma, Ltd",,3,1
7,Nov 15 2022 7:37AM,250771,20,"Exact-Rx, Inc.",,,9
8,Nov 15 2022 4:39AM,250769,10,ISDIN Corporation,,1,14
9,Nov 14 2022 6:13PM,250768,15,"Brookfield Pharmaceuticals, LLC",,,59


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 8:52AM,250782,19,"GCH Granules USA, Inc.",1,,
1,Nov 15 2022 8:42AM,250777,12,Hush Hush,17,,
2,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,62,,
3,Nov 15 2022 8:22AM,250775,10,ISDIN Corporation,2,,
4,Nov 15 2022 8:19AM,250774,10,ISDIN Corporation,1,,
5,Nov 15 2022 8:18AM,250773,10,ISDIN Corporation,3,,
6,Nov 15 2022 7:42AM,250772,19,"AdvaGen Pharma, Ltd",1,3,
7,Nov 15 2022 7:37AM,250771,20,"Exact-Rx, Inc.",9,,
8,Nov 15 2022 4:39AM,250769,10,ISDIN Corporation,14,1,
9,Nov 14 2022 6:13PM,250768,15,"Brookfield Pharmaceuticals, LLC",59,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 8:52AM,250782,19,"GCH Granules USA, Inc.",1,,
1,Nov 15 2022 8:42AM,250777,12,Hush Hush,17,,
2,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,62,,
3,Nov 15 2022 8:22AM,250775,10,ISDIN Corporation,2,,
4,Nov 15 2022 8:19AM,250774,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 8:52AM,250782,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Nov 15 2022 8:42AM,250777,12,Hush Hush,17.0,NaN,NaN
2,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,62.0,NaN,NaN
3,Nov 15 2022 8:22AM,250775,10,ISDIN Corporation,2.0,NaN,NaN
4,Nov 15 2022 8:19AM,250774,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 8:52AM,250782,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Nov 15 2022 8:42AM,250777,12,Hush Hush,17.0,0.0,0.0
2,Nov 15 2022 8:34AM,250776,10,ISDIN Corporation,62.0,0.0,0.0
3,Nov 15 2022 8:22AM,250775,10,ISDIN Corporation,2.0,0.0,0.0
4,Nov 15 2022 8:19AM,250774,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3259689,195.0,10.0,34.0
12,250777,17.0,0.0,0.0
15,501438,107.0,0.0,0.0
16,250766,1.0,0.0,0.0
19,1504437,8.0,4.0,23.0
20,501466,10.0,0.0,0.0
21,501431,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3259689,195.0,10.0,34.0
1,12,250777,17.0,0.0,0.0
2,15,501438,107.0,0.0,0.0
3,16,250766,1.0,0.0,0.0
4,19,1504437,8.0,4.0,23.0
5,20,501466,10.0,0.0,0.0
6,21,501431,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,195.0,10.0,34.0
1,12,17.0,0.0,0.0
2,15,107.0,0.0,0.0
3,16,1.0,0.0,0.0
4,19,8.0,4.0,23.0
5,20,10.0,0.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,195.0
1,12,Released,17.0
2,15,Released,107.0
3,16,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,34.0,0.0,0.0,0.0,23.0,0.0,0.0
Executing,10.0,0.0,0.0,0.0,4.0,0.0,1.0
Released,195.0,17.0,107.0,1.0,8.0,10.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,34.0,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,4.0,0.0,1.0
2,Released,195.0,17.0,107.0,1.0,8.0,10.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,34.0,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,4.0,0.0,1.0
2,Released,195.0,17.0,107.0,1.0,8.0,10.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()